In [ ]:
import glob
import pandas as pd
import dask.dataframe as dd
import psycopg2
import pandas as pd
from supabase import create_client, Client
from datetime import datetime, time
from datetime import datetime, time

In [ ]:
# para unificar todos los archivos en uno solo y poder trabajar sobre ese
files = glob.glob("./BD/*.csv")  
data=pd.concat((pd.read_csv(i) for i in files), ignore_index=True)
print(data)


In [ ]:
data.to_csv('base de datos 2014-2018.csv')

In [ ]:
#llevar las fechas a un mismo formato
df=dd.read_csv('base de datos 2014-2018.csv')
dates=[]
for i in df['FlightDate']:
    dates.append(datetime.strptime(i, "%Y-%m-%d"))
df['FlightDate']=dates
print(df['FlightDate'])

In [ ]:
b=dd.read_csv('base de datos 2014-2018.csv',dtype={
                     'CancellationCode': 'object',
                     'Div1Airport': 'object',
                     'Div1TailNum': 'object',
                     'Div2Airport': 'object',
                     'Div2TailNum': 'object',
                     'Div3Airport': 'object'
                 })

In [ ]:
# eliminar columnas innecesarias
base_datos=b.drop(columns=['DistanceGroup','ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk','DepDelayMinutes', 'DepDel15', 
                           'DepartureDelayGroups', 'DepTimeBlk','CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
                           'IATA_CODE_Reporting_Airline', "DivReachedDest","DivActualElapsedTime","DivArrDelay","DivDistance","Div1Airport",
                           "Div1AirportID","Div1AirportSeqID","Div1WheelsOn","Div1TotalGTime","Div1LongestGTime","Div1WheelsOff","Div1TailNum",

                           "Div2Airport","Div2AirportID","Div2AirportSeqID","Div2WheelsOn","Div2TotalGTime","Div2LongestGTime","Div2WheelsOff", 
                           "Div2TailNum","Div3Airport","Div3AirportID","Div3AirportSeqID","Div3WheelsOn","Div3TotalGTime","Div3LongestGTime",
                           "Div3WheelsOff","Div3TailNum","Div4Airport","Div4AirportID","Div4AirportSeqID","Div4WheelsOn","Div4TotalGTime",
                           "Div4LongestGTime","Div4WheelsOff","Div4TailNum","Div5Airport","Div5AirportID","Div5AirportSeqID","Div5WheelsOn",
                           "Div5TotalGTime","Div5LongestGTime","Div5WheelsOff","Div5TailNum", 'Tail_Number', 'Year',	'Quarter',	'Month', 'DayofMonth',	
                           'DayOfWeek', 'DivAirportLandings', 'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime', 'TaxiOut', 'TaxiIn'])

In [ ]:
print(base_datos.columns)

In [ ]:
# unificar todos los nombres de las variables
data=dd.read_csv('base de datos 2014-2018(1).csv')
names=['FL_DATE', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 
       'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 
       'ORIGIN_CITY_MARKET_ID', 'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 
       'ORIGIN_STATE_FIPS', 'ORIGIN_STATE_NM', 'ORIGIN_WAC', 'DEST_AIRPORT_ID', 
       'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 
       'DEST_STATE_ABR', 'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_WAC', 
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'WHEELS_OFF', 'WHEELS_ON', 
       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED', 'DIVERTED', 
       'AIR_TIME', 'FLIGHTS', 'DISTANCE']
data.columns=names
data.to_csv('base de datos 2014-2018(2).csv', index= False, single_file=True)


In [ ]:
csv_file_path = "base de datos 2014-2018(2).csv" 
df = dd.read_csv(csv_file_path) 
df.co
conn = psycopg2.connect(
    dbname="fligth_database",
    user="postgres",
    password="Katy+58029986",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# 3. Eliminar tablas existentes (si es necesario)
# cur.execute("DROP TABLE IF EXISTS aircrafts;")
# cur.execute("DROP TABLE IF EXISTS airports;")
#cur.execute("DROP TABLE IF EXISTS flights;")
#cur.execute("DROP TABLE IF EXISTS airlines;")

# 4. Crear las tablas necesarias
cur.execute("""
CREATE TABLE IF NOT EXISTS airlines (
    airline_id INT PRIMARY KEY,
    unique_carrier VARCHAR(10)
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS airports (
    airport_id INT,
    airport_seq_id INT,
    city_market_id VARCHAR(50),
    code VARCHAR(50),
    city_name VARCHAR(50),
    state_abr VARCHAR(50),
    state_fips VARCHAR(50),
    state_name VARCHAR(50),
    wac INT,
    PRIMARY KEY (airport_id, airport_seq_id)
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS flights (
    flight_id SERIAL PRIMARY KEY,
    fl_date DATE,
    op_unique_carrier VARCHAR(10),
    op_carrier_airline_id INT,
    op_carrier VARCHAR(10),
    op_carrier_fl_num VARCHAR(10),
    origin_airport_id INT,
    dest_airport_id INT,
    crs_dep_time TIME,
    dep_time TIME,
    dep_delay FLOAT,
    wheels_off TIME,
    wheels_on TIME,
    crs_arr_time TIME,
    arr_time TIME,
    arr_delay FLOAT,
    cancelled BOOLEAN,
    diverted BOOLEAN,
    air_time FLOAT,
    distance FLOAT,
    flights FLOAT
);
""")

conn.commit()
def convert_hhmm(time_val):
    """Convierte valores HHMM a objetos TIME."""
    try:
        if pd.isna(time_val) or time_val == '':
            return None
    
        time_num = int(float(time_val))
        
        if time_num == 2400:
            return time(23, 59, 59)
            
        if time_num < 0 or time_num > 2359:
            return None
        
        time_str = f"{time_num:04d}"
    
        hours = int(time_str[:2])
        minutes = int(time_str[2:])
        
        if hours > 23 or minutes > 59:
            return None
            
        return time(hours, minutes, 0)
    except (ValueError, TypeError):
        return None

def parse_fl_date(date_val):
    """Versión robusta para manejar múltiples formatos de fecha."""
    if pd.isna(date_val):
        return None
        
    try:
        parsed = pd.to_datetime(date_val, errors='coerce')
        if not pd.isna(parsed):
            return parsed.date()
            

        date_str = str(date_val).strip()
       
        if date_str.isdigit() and len(date_str) == 8:
            return datetime.strptime(date_str, "%Y%m%d").date()
            
        for fmt in ("%m/%d/%Y", "%Y-%m-%d", "%d-%m-%Y", "%m-%d-%Y"):
            try:
                return datetime.strptime(date_str, fmt).date()
            except ValueError:
                continue
                
        return None
        
    except Exception as e:
        print(f"Error parsing date {date_val}: {str(e)}")
        return None

airlines = df[['op_carrier_airline_id', 'op_unique_carrier']].drop_duplicates()
cur.executemany("""
    INSERT INTO airlines (airline_id, unique_carrier)
    VALUES (%s, %s)
    ON CONFLICT (airline_id) DO NOTHING;
""", [(row['op_carrier_airline_id'], row['op_unique_carrier']) for _, row in airlines.iterrows()])

origin_airports = df[['origin_airport_id', 'origin_airport_seq_id', 'origin_city_market_id',
                      'origin', 'origin_city_name', 'origin_state_abr', 'origin_state_fips',
                      'origin_state_nm', 'origin_wac']].drop_duplicates()

dest_airports = df[['dest_airport_id', 'dest_airport_seq_id', 'dest_city_market_id',
                    'dest', 'dest_city_name', 'dest_state_abr', 'dest_state_fips',
                    'dest_state_nm', 'dest_wac']].drop_duplicates()

origin_airports.columns = ['airport_id', 'airport_seq_id', 'city_market_id',
                           'code', 'city_name', 'state_abr', 'state_fips',
                           'state_name', 'wac']

dest_airports.columns = origin_airports.columns

airports = dd.concat([origin_airports, dest_airports]).drop_duplicates()

cur.executemany("""
    INSERT INTO airports (airport_id, airport_seq_id, city_market_id,
                          code, city_name, state_abr, state_fips,
                          state_name, wac)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (airport_id, airport_seq_id) DO NOTHING;
""", [tuple(row) for _, row in airports.iterrows()])

#Insertar los vuelos en `flights`
for _, row in df.iterrows():
    # Convertir todos los campos
    fl_date = parse_fl_date(row['fl_date'])
    
    crs_dep_time = convert_hhmm(row['crs_dep_time'])
    dep_time = convert_hhmm(row['dep_time'])
    wheels_off = convert_hhmm(row['wheels_off'])
    wheels_on = convert_hhmm(row['wheels_on'])
    crs_arr_time = convert_hhmm(row['crs_arr_time'])
    arr_time = convert_hhmm(row['arr_time'])   
    
    cur.execute("""
        INSERT INTO flights (fl_date, op_unique_carrier, op_carrier_airline_id,
                             op_carrier, op_carrier_fl_num,
                             origin_airport_id, dest_airport_id,
                             crs_dep_time, dep_time, dep_delay, wheels_off,
                             wheels_on, crs_arr_time, arr_time, arr_delay,
                             cancelled, diverted, air_time,
                             distance, flights)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s);
    """, (
        fl_date,
        row['op_unique_carrier'],
        row['op_carrier_airline_id'],
        row['op_carrier'],
        row['op_carrier_fl_num'],
        row['origin_airport_id'],
        row['dest_airport_id'],
        crs_dep_time,
        dep_time,
        row['dep_delay'],
        wheels_off,
        wheels_on,
        crs_arr_time,
        arr_time,
        row['arr_delay'],
        bool(row['cancelled']),
        bool(row['diverted']),
        row['air_time'],
        row['distance'],
        row['flights']
    ))

conn.commit()

cur.close()
conn.close()